# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [1]:

import warnings
warnings.simplefilter('ignore')

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [2]:
import pandas as pd
import sqlite3 as sql
import numpy as np

connection = sql.connect('../data/go_sales.sqlite')


Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [3]:
product = pd.DataFrame( pd.read_sql_query("SELECT * FROM product;", connection) )
product_type = pd.DataFrame( pd.read_sql_query("SELECT * FROM product_type;", connection) )
product_line = pd.DataFrame( pd.read_sql_query("SELECT * FROM product_line;", connection) )
sales_staff = pd.DataFrame( pd.read_sql_query("SELECT * FROM sales_staff;", connection) )
sales_branch = pd.DataFrame( pd.read_sql_query("SELECT * FROM sales_branch;", connection) )
retailer_site = pd.DataFrame( pd.read_sql_query("SELECT * FROM retailer_site;", connection) )
country = pd.DataFrame( pd.read_sql_query("SELECT * FROM country;", connection) )
order_details = pd.DataFrame( pd.read_sql_query("SELECT * FROM order_details;", connection) )
order_header = pd.DataFrame( pd.read_sql_query("SELECT * FROM order_header;", connection) )
target = pd.DataFrame( pd.read_sql_query("SELECT * FROM SALES_TARGETData;", connection) )
returned_item = pd.DataFrame( pd.read_sql_query("SELECT * FROM returned_item;", connection) )
return_reason = pd.DataFrame( pd.read_sql_query("SELECT * FROM return_reason;", connection) )

sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql_query(sql_query, connection)


,name
0,country
1,order_details
2,order_header
3,order_method
4,product
5,product_line
6,product_type
7,retailer_site
8,return_reason
9,returned_item


Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [ ]:
production_cost = product['PRODUCTION_COST'].astype(float)
has_cost = (production_cost > 50) & (production_cost < 100)

product.loc[has_cost, ['PRODUCT_NUMBER', 'PRODUCTION_COST']]

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [ ]:
product_margin = product['MARGIN'].astype(float)
has_margin = (product_margin < .2) | (product_margin > .6)

product.loc[(has_margin), ['PRODUCT_NUMBER', 'MARGIN']]

Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [ ]:
currency = country['CURRENCY_NAME']
country.loc[(currency == 'francs'), 'COUNTRY']

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [ ]:
product.loc[(product_margin > .5), 'INTRODUCTION_DATE'].drop_duplicates()

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 28 rijen)

In [ ]:
sales_branch.loc[
    (
        (sales_branch['ADDRESS2'].notna()) & 
        (sales_branch['REGION'].notna())
    )
, ['ADDRESS1', 'CITY']
]


Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [ ]:
country.loc[( (currency == 'dollars') |  (currency == 'new dollar') ), 'COUNTRY']

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [ ]:
sales_branch.loc[(sales_branch['POSTAL_ZONE'].str[0] == 'D'), ['ADDRESS1', 'ADDRESS2', 'CITY']]

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [ ]:
returned_item['RETURN_CODE'].count()

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [ ]:
retailer_site['REGION'].count()

Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [ ]:
min_margin = product_margin.min()
max_margin = product_margin.max()
avg_margin = product_margin.mean()

pd.DataFrame({
    'min': min_margin,
    'max': max_margin,
    'avg': avg_margin
}, index=[''])

Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [ ]:
sales_branch['ADDRESS2'].isna().count()

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [ ]:
has_korting = order_details['UNIT_SALE_PRICE'].astype(float) < order_details['UNIT_PRICE'].astype(float)
order_details.loc[(has_korting), 'UNIT_COST'].astype(float).mean()


Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [ ]:
sales_staff.groupby('POSITION_EN', as_index=False)['SALES_STAFF_CODE'].count()


Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [ ]:
work_phone = sales_staff.groupby('WORK_PHONE')['SALES_STAFF_CODE'].count()
work_phone[work_phone > 4]


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [ ]:
sales_country  = pd.merge(retailer_site, country, 'outer')
sales_country.loc[(sales_country['COUNTRY'] == 'Netherlands'), ['ADDRESS1', 'CITY']]



Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [ ]:
product_pdtype = pd.merge(product, product_type)
product_pdtype.loc[(product_pdtype['PRODUCT_TYPE_EN'] == 'Eyewear'), 'PRODUCT_NAME']

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [3]:
retailer_header = pd.merge(retailer_site, order_header)
retailer_header_staff = pd.merge(retailer_header, sales_staff)
has_manager_position = retailer_header_staff['POSITION_EN'] == 'Branch Manager'

retailer_header_staff.loc[(has_manager_position),  ['ADDRESS1', 'FIRST_NAME', 'LAST_NAME']].drop_duplicates()

,ADDRESS1,FIRST_NAME,LAST_NAME
4176,"Alameda Santos, 9876",Bayard,Lopes


Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

In [7]:
staff_header = pd.merge(sales_staff, order_header, how = 'left')
is_manager = staff_header['POSITION_EN'].str.contains('Manager')
staff_header.loc[is_manager, ['POSITION_EN', 'ORDER_DATE']].drop_duplicates()


,POSITION_EN,ORDER_DATE
421,Branch Manager,NaN
422,General Manager,NaN
1276,Regional Manager,NaN
1643,District Manager,NaN
5325,Branch Manager,2022-09-09
5326,Branch Manager,2022-10-31
5327,Branch Manager,2022-12-13
5328,Branch Manager,2022-03-13
5329,Branch Manager,2022-01-09


Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [ ]:
more_then_750 = order_details.loc[(order_details['QUANTITY'] > 750)]
more_product = pd.merge(product, more_then_750)
more_product_type = pd.merge(more_product, product_type)

more_product_type.loc[:, ['PRODUCT_NAME', 'PRODUCT_TYPE_EN']].drop_duplicates()

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [ ]:
unit_price = order_details['UNIT_PRICE'].astype(float)
unit_sale_price = order_details['UNIT_SALE_PRICE'].astype(float)
korting = (unit_price - unit_sale_price) / unit_price
korting_products = pd.merge(order_details.loc[korting > .4], product)

korting_products.loc[:, 'PRODUCT_NAME'].drop_duplicates()


Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [ ]:
order_return = pd.merge(order_details, returned_item)
return_quantity = order_return['RETURN_QUANTITY'].astype(int)
quantity = order_return['QUANTITY'].astype(int)
returned_items = order_return.loc[return_quantity / quantity > .9]
pd.merge(returned_items, return_reason).loc[:, 'RETURN_DESCRIPTION_EN'].drop_duplicates()


## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [28]:
product_types = pd.merge(product, product_type)
product_types.groupby('PRODUCT_TYPE_EN')['PRODUCT_NUMBER'].count()



PRODUCT_TYPE_EN
Binoculars               4
Climbing Accessories     7
Cooking Gear            10
Eyewear                  5
First Aid                5
Golf Accessories         4
Insect Repellents        5
Irons                    4
Knives                   5
Lanterns                12
Navigation               4
Packs                    6
Putters                  3
Rope                     4
Safety                   4
Sleeping Bags            7
Sunscreen                5
Tents                    6
Tools                    6
Watches                  5
Woods                    4
Name: PRODUCT_NUMBER, dtype: int64

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [27]:
country_retail = pd.merge(country, retailer_site)
country_retail.groupby('COUNTRY', as_index=False)['RETAILER_CODE'].count()

,COUNTRY,RETAILER_CODE
0,Australia,10
1,Austria,10
2,Belgium,10
3,Brazil,1
4,Canada,31
5,China,10
6,Denmark,6
7,Finland,5
8,France,29
9,Germany,32


Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [62]:
product_pdtype = pd.merge(product, product_type)
product_pdtype.loc[product_pdtype['PRODUCT_TYPE_EN'] == 'Cooking Gear']
pd_header = pd.merge(product_pdtype,order_details)

pd_header['QUANTITY'] = pd_header['QUANTITY'].astype(int)
pd_header['UNIT_SALE_PRICE'] = pd_header['UNIT_SALE_PRICE'].astype(float)


pd_header.groupby('PRODUCT_NAME').agg({
    'QUANTITY': 'sum',
    'UNIT_SALE_PRICE': 'mean'
}).sort_values(by='QUANTITY', ascending=False).reset_index()




,PRODUCT_NAME,QUANTITY,UNIT_SALE_PRICE
0,BugShield Extreme,69996,6.300222
1,Sun Shelter 30,69004,4.562563
2,BugShield Lotion Lite,54928,6.397548
3,Sun Shield,53086,5.564401
4,BugShield Lotion,51126,6.502539
...,...,...,...
110,Lady Hailstorm Steel Irons,1962,494.285765
111,Lady Hailstorm Titanium Woods Set,1808,1121.165079
112,Course Pro Golf Bag,1746,196.612297
113,Lady Hailstorm Steel Woods Set,1506,845.718743


Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

In [74]:
staff_target = pd.merge(sales_staff,target)
staff_target_branch = pd.merge(staff_target,sales_branch)
staff_target_branch = staff_target_branch.loc[staff_target_branch['CITY'] == 'Amsterdam']
staff_target_branch.groupby(['FIRST_NAME', 'LAST_NAME']).agg({
    'SALES_TARGET': ['min', 'max']
})

SALES_TARGET      
                                     min   max
FIRST_NAME LAST_NAME                          
Belinda    Jansen-Velasquez         1004   984
Ellen      Shapiro                   100   997
Jan        Haverkamp                1001  9987
Maria      Laponder                 1000  9993

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

In [61]:
country_branch = pd.merge(country,sales_branch)
country_branch_staff = pd.merge(country_branch, sales_staff)

country_branch_staff.rename(columns={'CITY': 'verkoper', 'SALES_STAFF_CODE': 'klanten'}, inplace=True)

# Grouping and aggregating
table = country_branch_staff.groupby(['COUNTRY', 'verkoper']).agg({
    'klanten': 'count'
})


table.reset_index()


,COUNTRY,verkoper,klanten
0,Australia,Melbourne,5
1,Austria,Wien,2
2,Belgium,Heverlee,4
3,Brazil,São Paulo,3
4,Canada,Calgary,5
5,Canada,Toronto,5
6,China,Shanghai,2
7,Finland,Kuopio,2
8,France,Lyon,4
9,France,Paris,4


## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

In [60]:
sold_something = order_header['SALES_STAFF_CODE'].unique()

for index, row in sales_staff.iterrows():
    code = row['SALES_STAFF_CODE']
    sales_staff.at[index, 'SOLD_SOMETHING'] = code in sold_something 

sales_staff.loc[sales_staff['SOLD_SOMETHING'] == False, ['FIRST_NAME', 'LAST_NAME']].reset_index()


,index,FIRST_NAME,LAST_NAME
0,7,Walter,Taylor
1,8,Penelope,Tamrine
2,11,Polly,Linton
3,19,Gracy,Gellens
4,26,Else,Mörike
5,27,Frank,Fuchs
6,30,Fritz,Hirsch
7,31,Jörg,Kunze
8,33,Maria,Iacobucci
9,38,Kick,Kalkman


Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

In [75]:
margins = product['MARGIN'].astype(float)
avg_margin = margins.mean()
lower_than_average = product.loc[margins < avg_margin, 'PRODUCT_NUMBER'].count()

pd.DataFrame({
    'PRODUCTS': [lower_than_average], 
    'avg_margin': [avg_margin]
    })


,PRODUCTS,avg_margin
0,59,0.396609


Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

In [194]:
over_500 = order_details.loc[order_details['UNIT_SALE_PRICE'].astype(float) > 500]

for index, row in product.iterrows():
    number = row['PRODUCT_NUMBER']
    product.at[index, 'RETURNED'] = number not in pd.merge(order_details,returned_item)['PRODUCT_NUMBER'].unique()
    product.at[index, 'OVER_500'] = number in over_500['PRODUCT_NUMBER'].unique()

product.loc[(product['RETURNED'] == False) & (product['OVER_500'] == True), 'PRODUCT_NAME'].reset_index()


,index,PRODUCT_NAME
0,3,Hailstorm Steel Irons
1,4,Hailstorm Titanium Irons
2,5,Lady Hailstorm Steel Irons
3,6,Lady Hailstorm Titanium Irons
4,7,Hailstorm Titanium Woods Set
5,9,Lady Hailstorm Titanium Woods Set
6,10,Lady Hailstorm Steel Woods Set
7,19,Star Dome
8,20,Star Gazer 2
9,21,Star Gazer 3


Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

In [4]:
staff = sales_staff.loc[:, ['LAST_NAME', 'POSITION_EN']]

for i,row in staff.iterrows():
    position = row['POSITION_EN']
    staff.at[i, 'IS_MANAGER'] = 'Manager' in position

staff.loc[:, ['LAST_NAME', 'IS_MANAGER']]

,LAST_NAME,IS_MANAGER
0,Savolainen,False
1,Kim,False
2,Choi,False
3,Kao,False
4,Fang,False
...,...,...
97,Iglesias,False
98,Lopes,True
99,Barros,False
100,Guimarães,False


Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [ ]:
from datetime import date
date.today().year

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [ ]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 

In [7]:
from datetime import datetime, date
date_format = '%d-%m-%Y'

for i, row in sales_staff.iterrows():
    years = date.today().year - datetime.strptime(row['DATE_HIRED'], date_format).year
    
    if years >= 12:
        sales_staff.at[i, 'IN_DIENST'] = 'Lang in dienst'
    else:
        sales_staff.at[i, 'IN_DIENST'] = 'Kort in dienst'

sales_staff.loc[:, ['SALES_STAFF_CODE', 'IN_DIENST']]

,SALES_STAFF_CODE,IN_DIENST
0,100,Kort in dienst
1,101,Kort in dienst
2,102,Kort in dienst
3,103,Kort in dienst
4,104,Lang in dienst
...,...,...
97,95,Kort in dienst
98,96,Kort in dienst
99,97,Kort in dienst
100,98,Kort in dienst
